In [24]:
import ipywidgets as ipyw

In [10]:
import pandas as pd
import seaborn as sns
import numpy as np

In [11]:
titanic = sns.load_dataset("titanic")
df = titanic

In [90]:
titanic[["age", "pclass", "fare", "survived"]]

,age,pclass,fare,survived
0,22.0,3,7.2500,0
1,38.0,1,71.2833,1
2,26.0,3,7.9250,1
3,35.0,1,53.1000,1
4,35.0,3,8.0500,0
...,...,...,...,...
886,27.0,2,13.0000,0
887,19.0,1,30.0000,1
888,NaN,3,23.4500,0
889,26.0,1,30.0000,1


Fill value should accept 0 and "N/A"
Marginsname : text area

In [93]:
col_list = list(df)

func_w_dict = {
    "values":       (ipyw.SelectMultiple, {"options":col_list}),#, "description":"Column selector"}),
    "index":        (ipyw.SelectMultiple, {"options":col_list}),#, "description":"Column selector"}),
    "columns":      (ipyw.SelectMultiple, {"options":col_list}),#, "description":"Column selector"}),
    "aggfunc":      (ipyw.SelectMultiple, {"options":{"mean":np.mean, "sum":np.sum, "min":min, "max":max, "count":np.count_nonzero, "std":np.std, "len":len}}),
    #"fill_value":   
    "margins":      (ipyw.Checkbox,       {"value":False}),
    "dropna":       (ipyw.Checkbox,       {"value":True}),
    #"margins_name":       
    "observed":     (ipyw.Checkbox,       {"value":False}),
}

def widget_dispenser(widget_name):
    widget, kwargs = func_w_dict[widget_name]
    return widget(**kwargs)

widget_dispenser('values')

SelectMultiple(options=('survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'class', 'wh…

```python
pd.pivot_table(data,
    values=None,         # values: column to aggregate, optional
    index=None,          # index: column, Grouper, array, or list of the previous
    columns=None,        # columns: column, Grouper, array, or list of the previous
    aggfunc='mean',      # aggfunc: function, list of functions, dict, default numpy.mean
    fill_value=None,     # fill_value: scalar, default None
    margins=False,       # margins: bool, default False
    dropna=True,         # dropna: bool, default True
    margins_name='All',  # margins_name: str, default ‘All’
    observed=False       # observed: bool, default False
)
```

In [98]:
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as ipyw
import pandas as pd

from IPython.display import display, clear_output


class PlotUI(ipyw.HBox):
    
    def __init__(self, df):
        if not isinstance(df, pd.DataFrame):
            raise ValueError
        super().__init__()
        self.df = df
            
        self.widgets_names = []
        self.widgets_list  = []
        self.wdict = {}

        # create a VBox and Output widgets
        vbox = ipyw.VBox()
        hbox = ipyw.HBox()
        self.output = ipyw.Output()

        # columns selector
        #self.cols_selector = wdispenser("a_select")
        #self.cols_selector.value = [f for f in self.df]
        
        # create widgets and add them to the HBox
        for arg_name, widget_name in func_w_dict.items():
            # create widget
            w = widget_dispenser(arg_name)
            
            self.widgets_names.append(arg_name)
            cb = self._create_enable_checkbox(descr=arg_name)
            self._link_enable_status(w, cb)
            self.wdict[arg_name] = (cb, w)
            
            box = [cb, w]
            
            self.widgets_list.append(ipyw.HBox(box))
        
        #self._set_init_values_widgets()
        
        self.connect_widgets()
        
        # set the widgets in a VBox
        vbox.children = [*self.widgets_list]
        
        # wrap all the UI for the plot in a hbox
        self.children = [vbox, self.output]

        
        
    def _create_enable_checkbox(self, descr="Disable:"):
        """Create a checkbox"""
        return ipyw.Checkbox(description=descr, 
                             value=True, 
                             indent=False, 
                             layout=ipyw.Layout(width="100px"))
    
    
    def _link_enable_status(self, w, cb):
        """Link checkbox values to widget disabled value"""
        return ipyw.link((cb, "value"), (w, "disabled"))


    def connect_widgets(self):
        """Observe widgets values and checbox to trigger a replot"""
        for wbox in self.widgets_list:
            children = wbox.children
            children[1].observe(self.display_plot, 'value')    
            children[0].observe(self.display_plot, "value")
        # change in sub cols
        #self.cols_selector.observe(self.display_plot, "value")
            
            
    def retrieve_enabled_kwargs(self):
        """Retrieve values of args that are enabled into a dict"""
        kwargs = {}
        for wbox, wname in zip(self.widgets_list, self.widgets_names):
            children = wbox.children
            if children[0].value == False:
                kwargs[wname] = children[1].value   
        return kwargs
    
        
    def display_plot(self, *_):
        # clear_output de IPython.display
        with self.output:
            clear_output(wait=True)
            kwargs = self.retrieve_enabled_kwargs()
            kwargs_str = self._format_kwargs(kwargs)
            display(f"pd.pivot_table({kwargs_str})")
            # get all widgets names and values in a dict
            try:
                res = pd.pivot_table(self.df, **kwargs)
                # print(res)
                display(res)
                #res.plot()
            except Exception as e:
                print(e)
            
            
    def _format_kwargs(self, kwargs):
        if not bool(kwargs):
            return ""
        def format_arg_on_type(arg):
            if isinstance(arg, str):
                return f"'{arg}'"
            return f"{arg}"
        return ", ".join([f"{t[0]}={format_arg_on_type(t[1])}" for t in kwargs.items()])
    


In [99]:
PlotUI(titanic)

PlotUI(children=(VBox(children=(HBox(children=(Checkbox(value=True, description='values', indent=False, layout…

In [12]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [17]:
pd.pivot_table(df, 
               values=['fare', 'age'],
               index=['sex', 'class'],
               aggfunc={'fare': np.sum,
                        'age': np.mean})

age       fare
sex    class                       
female First   34.611765  9975.8250
       Second  28.722973  1669.7292
       Third   21.750000  2321.1086
male   First   41.281386  8201.5875
       Second  30.740707  2132.1125
       Third   26.507589  4393.5865

In [23]:
pd.pivot_table(df, 
               values=['fare', 'age'],
               index=['sex', 'class'],
               aggfunc={'fare': np.sum,
                        'age': np.mean},
              margins=True)

age        fare
sex    class                        
female First   34.611765   9975.8250
       Second  28.722973   1669.7292
       Third   21.750000   2321.1086
male   First   41.281386   8201.5875
       Second  30.740707   2132.1125
       Third   26.507589   4393.5865
All            29.699118  24771.8830

In [22]:
table = pd.pivot_table(df,
               values='fare',
               index=['who', 'class'],
               columns=['alive'],
               aggfunc=np.sum,
               fill_value=0)
table

alive                no        yes
who   class                       
child First    151.5500   684.7458
      Second     0.0000   538.1542
      Third    957.9959   388.7751
man   First   4842.9081  3005.2711
      Second  1773.4958   112.8625
      Third   3032.9366   584.5915
woman First    180.2625  9312.6750
      Second   109.5000  1267.8292
      Third   1094.0710   656.3250

In [19]:
pd.pivot_table(df,
               values=['fare', 'age'],
               index=['who', 'class'],
               aggfunc={'fare': np.mean,
                        'age': [min, max, np.mean]})

age                          fare
               max       mean    min        mean
who   class                                     
child First   15.0   7.820000   0.92  139.382633
      Second  14.0   4.543684   0.67   28.323905
      Third   15.0   6.817586   0.42   23.220190
man   First   80.0  42.382653  17.00   65.951086
      Second  70.0  33.588889  16.00   19.054124
      Third   74.0  28.995556  16.00   11.340213
woman First   63.0  35.500000  16.00  104.317995
      Second  57.0  32.179688  17.00   20.868624
      Third   63.0  27.854167  16.00   15.354351